In [ ]:
#dependencies 
#had to 
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
from coincallandwritepythonfile import refined_coin_list, write_to_storage

In [ ]:
#make sure no blank id values for query
refined_coin_list = refined_coin_list[refined_coin_list['id'] != ""]
#get ids for each coin and convert to list
coin_ids = refined_coin_list.id.to_list()

In [ ]:
schedule.every(15).seconds.do(func)

In [1]:

import schedule
import time
  
def func():
    print("Geeksforgeeks")

schedule.every(10).seconds.do(func)
#schedule.run_all(delay_seconds= 3).do(func)
count = 0
while True and count <= 3:
    schedule.run_pending()
    time.sleep(10)
    print(count)
    count +=1
    
     

0
Geeksforgeeks
1
Geeksforgeeks
2
Geeksforgeeks
3


In [ ]:

import schedule
import time
  
def func():
    print("Geeksforgeeks")
  
#schedule.every(1).minutes.do(func)
schedule.run_all(delay_seconds= 3).do(func)

count = 0
while count < 6:
    schedule.run_pending()
    time.sleep(1)
    count += 1

In [ ]:
def get_coin_info(list_of_coins_, api_key):
    """Function makes API call for each coin ID within list of coins.
    Designed to take list from top 100 coins and get their most recent price information. """

    #create an empty dataframe that we will append responses too
    df = pd.DataFrame()
    #a count variable to index the respones when trying to concat 
    count =0
    for id in list_of_coins_:
        url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
        parameters = {
            'id': id,
                    }
        headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key
        }

        #make API call and return response as a dictionary
        try:
            session = Session()
            session.headers.update(headers)
            response = session.get(url, params=parameters)
            data = json.loads(response.text)
            #erroring on 'Data' sometimes, like a rogue id
        except KeyError:
            continue
        
        #parse dict and load in relavant data to a DF
        cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')
         
        #expand nested quote column containing price information
        expanded_df = pd.DataFrame.from_dict(cmc_df['quote'][0], orient= 'index')     
        #set then index for count ie the number of runs 
        expanded_df.set_index(pd.Index([count]),inplace= True)

        #dropping nested columns that we won't explore during this exercise
        trial = cmc_df.drop(['quote','tags', 'platform'], axis= 1)
          
        #setting index as count variable so can join back
        trial.set_index(pd.Index([count]),inplace= True)
        
        #concat was tricky had to add
        #had to add key to get them to append properly
        coin_price_df = pd.concat([trial,expanded_df], axis=1, keys=['trial', 'expanded_df']  )
        #coin_price_df.reset_index(drop = True, inplace= True)
        coin_price_merge = pd.merge(trial, expanded_df)
        #this saves it as a coreframe

               
        df = df.append(coin_price_merge)        

        count +=1
        #df.set_index(pd.Index([count]),inplace= True)
    #had to drop level inorder to get ride of multilevel index   
    # #https://sparkbyexamples.com/pandas/pandas-multiindex-dataframe-examples/?msclkid=fba45bb6c10e11eca1d376f5b274bab7 
        
    #df = df.droplevel(0, axis =1)
    return df

        


In [ ]:
df = get_coin_info(coin_ids[:15],  config.coin_api_key)

In [ ]:
#write to storage function takes dataset input as a CSV so had to convert
df = df.to_csv(index = False )

In [ ]:
#write the new DF to storage
storage_account_name = config.storage_account_name
storage_account_key = config.storage_account_key
container_name = "coinprice"
directory_name = "raw"

dataframe = write_to_storage(storage_account_name=storage_account_name, storage_account_key= storage_account_key, container_name= container_name, directory_name= directory_name,\
    dataset = df, file_name= "coinprice.csv")